In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split  

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier # for adapvtive boosting

In [3]:
bank = pd.read_csv('C:/Users/Administrator/Desktop/ds/bank-full.csv', sep = ';')
bank.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
bank['y'].value_counts()/bank.shape[0]*100

no     88.30152
yes    11.69848
Name: y, dtype: float64

In [5]:
bank_dummy      = pd.get_dummies(bank.drop('y',axis = 1))
bank_dummy['y'] = bank['y']
bank_dummy.shape


(45211, 52)

In [6]:
train,test = train_test_split(bank_dummy,test_size=0.3,
                             random_state = 100)
print(train.shape)
print(test.shape)
train.head()
#saggegate
train_y = train['y']
test_y  = test ['y']
train_x = train.drop('y', axis = 1)#axis 1 is for row wise operation
test_x  = test.drop ('y', axis = 1)
train_x.shape

(31647, 52)
(13564, 52)


(31647, 51)

In [48]:
#model
model = DecisionTreeClassifier(random_state = 100)
model
model.fit(train_x, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [49]:
test_pred = model.predict(test_x)
print(len(test_pred))
test_pred[:5]
#dataframe
df_bank = pd.DataFrame({'actual':test_y,
                        'predicted': test_pred})
df_bank.head()
df_bank['status'] = df_bank['actual']==df_bank['predicted']
df_bank.head()
#calculating
df_bank['status'].value_counts()/df_bank.shape[0]*100
df_bank.head(2)

13564


,actual,predicted,status
14789,no,no,True
8968,no,no,True


In [50]:
#MANUALLY CHECKING 
tp_dt = df_bank[(df_bank['predicted']=='yes') & (df_bank['actual']=='yes')].shape[0]
tn_dt = df_bank[(df_bank['predicted']=='no') & (df_bank['actual']=='no')].shape[0]
fp_dt = df_bank[(df_bank['predicted']=='yes') & (df_bank['actual']=='no')].shape[0]
fn_dt = df_bank[(df_bank['predicted']=='no') & (df_bank['actual']=='yes')].shape[0]
print(tp_dt,tn_dt,fp_dt,fn_dt)

775 11124 861 804


In [65]:
acc_dt = (tp_dt + tn_dt)/(tp_dt+tn_dt+fp_dt+fn_dt)
print(acc_dt)
sensitivity_dt = tp_dt/(tp_dt+fn_dt)
print(sensitivity_dt)
specificity_dt = tn_dt/(tn_dt+fp_dt)
print(specificity_dt)

0.8772485992332645
0.4908169727675744
0.9281602002503129


# Random Forest


In [52]:
model_rf = RandomForestClassifier(random_state=100)
model_rf.fit(train_x,train_y)

test_pred_rf = model_rf.predict(test_x)
df_rf   = pd.DataFrame({'actual': test_y,
                         'predicted': test_pred_rf})
df_rf['pred_status'] = df_rf['actual'] == df_rf['predicted']
df_rf['pred_status'].sum()/df_rf.shape[0]*100

89.97345915659098

In [53]:
tp_rf = df_rf[(df_rf['predicted']=='yes') & (df_rf['actual']=='yes')].shape[0]
tn_rf = df_rf[(df_rf['predicted']=='no') & (df_rf['actual']=='no')].shape[0]
fp_rf = df_rf[(df_rf['predicted']=='yes') & (df_rf['actual']=='no')].shape[0]
fn_rf = df_rf[(df_rf['predicted']=='no') & (df_rf['actual']=='yes')].shape[0]
print(tp_rf,tn_rf,fp_rf,fn_rf)

561 11643 342 1018


In [66]:
acc_rf = (tp_rf + tn_rf)/(tp_rf+tn_rf+fp_rf+fn_rf)
print(acc_rf)
sensitivity_rf = tp/(tp_rf+fn_rf)
print(sensitivity_rf)
specificity_rf = tn_rf/(tn_rf+fp_rf)
print(specificity_rf)

0.8997345915659097
0.37048765041165294
0.9714643304130163


# Adaptive boost

In [55]:
model = AdaBoostClassifier(random_state=100)
model.fit(train_x,train_y)
test_pred_abm = model.predict(test_x)
df_abm  = pd.DataFrame({'actual': test_y,
                         'predicted': test_pred_abm})


In [56]:
#MANUALLY CHECKING 
tp_abm = df_abm[(df_abm['predicted']=='yes') & (df_abm['actual']=='yes')].shape[0]
tn_abm = df_abm[(df_abm['predicted']=='no') & (df_abm['actual']=='no')].shape[0]
fp_abm = df_abm[(df_abm['predicted']=='yes') & (df_abm['actual']=='no')].shape[0]
fn_abm = df_abm[(df_abm['predicted']=='no') & (df_abm['actual']=='yes')].shape[0]
print(tp_abm,tn_abm,fp_abm,fn_abm)

585 11585 400 994


In [69]:
acc_abm = (tp_abm + tn_abm)/(tp_abm+tn_abm+fp_abm+fn_abm)
print(acc_rf)
sensitivity_abm = tp_abm/(tp_abm+fn_abm)
print(sensitivity_rf)
specificity_abm = tn_abm/(tn_abm+fp_abm)
print(specificity_rf)

0.8997345915659097
0.37048765041165294
0.9714643304130163


In [70]:
s_abm = pd.Series({'TP': tp_abm, 'TN' : tn_abm, 'FP': fp_abm, 'FN': fn_abm, 'ACCURACY': acc_abm,'SENSITIVITY': sensitivity_abm,
                  'SPECIFICITY': specificity_abm})

In [71]:
s_abm

TP               585.000000
TN             11585.000000
FP               400.000000
FN               994.000000
ACCURACY           0.897228
SENSITIVITY        0.370488
SPECIFICITY        0.966625
dtype: float64

In [72]:
s_rf = pd.Series({'TP': tp_rf, 'TN' : tn_rf, 'FP': fp_rf, 'FN': fn_rf, 'ACCURACY': acc_rf,'SENSITIVITY': sensitivity_rf,
                  'SPECIFICITY': specificity_rf})

In [73]:
s_rf

TP               561.000000
TN             11643.000000
FP               342.000000
FN              1018.000000
ACCURACY           0.899735
SENSITIVITY        0.370488
SPECIFICITY        0.971464
dtype: float64

In [74]:
s_dt = pd.Series({'TP': tp_dt, 'TN' : tn_dt, 'FP': fp_dt, 'FN': fn_dt, 'ACCURACY': acc_dt,'SENSITIVITY': sensitivity_dt,
                  'SPECIFICITY': specificity_dt})
s_dt

TP               775.000000
TN             11124.000000
FP               861.000000
FN               804.000000
ACCURACY           0.877249
SENSITIVITY        0.490817
SPECIFICITY        0.928160
dtype: float64

In [75]:
df_complete = pd.DataFrame({'Decision Tree': s_dt, 'Random Forest': s_rf, 'Adaptive Boosting': s_abm})
df_complete

,Decision Tree,Random Forest,Adaptive Boosting
TP,775.000000,561.000000,585.000000
TN,11124.000000,11643.000000,11585.000000
FP,861.000000,342.000000,400.000000
FN,804.000000,1018.000000,994.000000
ACCURACY,0.877249,0.899735,0.897228
SENSITIVITY,0.490817,0.370488,0.370488
SPECIFICITY,0.928160,0.971464,0.966625
